<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка данных</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Обучение-разных-моделей-в-пайплайне" data-toc-modified-id="Обучение-разных-моделей-в-пайплайне-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обучение разных моделей в пайплайне</a></span></li><li><span><a href="#Анализ-моделей" data-toc-modified-id="Анализ-моделей-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Анализ моделей</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Тестирование</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

**Наша задача**

Обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Для выполнения задачи необходимо построить модель со значением метрики качества *F1* не меньше 0.75.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

**План работы будет таким:**

1. Загрузить данные и подготовить их.
2. Обучить разные модели с различными гиперпараметрами.
3. Сделать выводы.
  
Прежде чем загружать датасеты, нужно провести импорт необходимых библиотек.

In [ ]:
import numpy as np
import pandas as pd
import re
import spacy
from tqdm.notebook import tqdm
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier

RANDOM_STATE = 12345

Кроме того, напишем функции, которые пригодятся нам в дальнейшей работе.

In [ ]:
#Функция получения общей информации о датасете, о дубликатах и пропусках.
def information(df):
    df.info();
    print('\nКоличество дубликатов:',df.duplicated().sum(),'\n')
    print('Количество пропусков:\n',df.isna().sum())

In [ ]:
# Функция для очистки текста
def clean_text(text):
    # приводим текст к нижнему регистру
    text = text.lower()
    # создаем регулярное выражение для удаления лишних символов
    regular = r'[\*+\#+\№\"\-+\+\=+\?+\&\^\.+\;\,+\>+\(\)\/+\:\\+]'
    # регулярное выражение для замены ссылки на "URL"
    regular_url = r'(http\S+)|(www\S+)|([\w\d]+www\S+)|([\w\d]+http\S+)'
    # удаляем лишние символы
    text = re.sub(regular, '', text)
    # заменяем ссылки на "URL"
    text = re.sub(regular_url, r'URL', text)
    # заменяем числа и цифры на ' NUM '
    text = re.sub(r'(\d+\s\d+)|(\d+)',' NUM ', text)
    # удаляем лишние пробелы
    text = re.sub(r'\s+', ' ', text)
    # возвращаем очищенные данные
    return text

## Загрузка и подготовка данных

### Загрузка данных

In [ ]:
try:
    data = pd.read_csv('???/toxic_comments.csv')
except:
    data = pd.read_csv('https://???/toxic_comments.csv')

Посмотрим первые строки датасета.

In [ ]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Сразу удалим столбец 'Unnamed: 0', он явно лишний.

In [ ]:
data = data.drop(['Unnamed: 0'], axis=1)

Теперь посмотрим общую информацию о датасете.

In [ ]:
information(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB

Количество дубликатов: 0 

Количество пропусков:
 text     0
toxic    0
dtype: int64


***Вывод:*** Загрузили датасет, общая информация показала, что в нем 152292 текста, дубликатов и пропусков нет. Тексты на английском языке, это надо будет учесть при дальнейшей подготовке данных к обучению.

### Подготовка данных

 Прежде чем начинать обработку текстов для обучения посмотрим, есть ли дисбаланс классов в целевом признаке.

In [ ]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Видно, что токсичных комментариев всего 10% от всего датасета. Чтобы уменьшить влияние дисбаланса, обучение будем проводить, используя кросс-валидацию.

Для подготовки данных к обучению нам нужно сначала очистить тексты от лишней информации (лишние символы, цифры, ссылки, пробелы) с помощью функции, написанной выше, затем токенизировать тексты, удалить стоп-слова и лемматизировать слова. Результат запишем в новую колонку.

In [ ]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
def lemmatize_text(text):
    text = clean_text(text)
    doc = nlp(text)
    text = [token.lemma_ for token in doc]
    text = ' '.join(text)
    return text

In [ ]:
tqdm.pandas()

data['lemm_text'] = data['text'].progress_apply(lemmatize_text)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [ ]:
data.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,d'aww ! he match this background colour I be s...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man I be really not try to edit war it be ...
3,"""\nMore\nI can't make any real suggestions on ...",0,more I can not make any real suggestion on i...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


Теперь разобьем датасет на тренировочную и тестовую выборки, не забудем добавить стратификацию, так как дисбаланс классов хоть и уменьшился, но не исчез совсем.

In [ ]:
X = data.drop(['text','toxic'], axis=1)
y = data['toxic']
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    random_state=RANDOM_STATE, test_size=0.1, stratify=y)

In [ ]:
X_train = X_train['lemm_text']
X_test = X_test['lemm_text']

***Вывод:*** На этапе подготовки данных мы проанализировали дисбаланс классов целевого признака, обнаружили его, но будем бороться с ним на этапе кросс-валидации. Кроме того, провели обработку текстов: очистили от лишней информации, провели лемматизацию. Затем разбили датасет на тренировочную и тестовую выборки. Можно переходить к обучению моделей.

## Обучение

### Обучение разных моделей в пайплайне

Чтобы сравнить несколько моделей, удобно использовать пайплайн с перебором разных гиперпараметров в GridsearchCV. Целевой признак у нас один - Toxic. Вариантов у него всего 2 - да или нет (1 или 0). А значит нам предстоит задача бинарной классификации. Для бинарной классификации нам подойдут модели логистической регрессии, KNN и LightGBM (для всех, кроме LightGBM, используем перебор разных гиперпараметров). Оценку моделей будем проводить метрикой F1. Так же включим в пайплайн получение признаков с помощью TF-IDF.

In [ ]:
pipe_final = Pipeline(
        [
            ('vect', TfidfVectorizer()),
            ('models', LogisticRegression(random_state=RANDOM_STATE))
        ]
    )

In [ ]:
param_grid = [

    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(2, 5),
    },

    {
        'models': [LogisticRegression(random_state=RANDOM_STATE, solver='liblinear')],
        'models__penalty': ['l1','l2'],
        'models__C': range(5, 15)
    },

    {
        'models': [LGBMClassifier(random_state=RANDOM_STATE, n_estimators=50)]

    }
]

Запустим обучение перебором, используя метод GridSearchCV.

In [ ]:
grid = GridSearchCV(
    pipe_final,
    param_grid=param_grid,
    cv=3,
    scoring='f1',
    n_jobs=1
)
grid.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                       ('models',
                                        LogisticRegression(random_state=12345))]),
             n_jobs=1,
             param_grid=[{'models': [KNeighborsClassifier()],
                          'models__n_neighbors': range(2, 5)},
                         {'models': [LogisticRegression(random_state=12345,
                                                        solver='liblinear')],
                          'models__C': range(5, 15),
                          'models__penalty': ['l1', 'l2']},
                         {'models': [LGBMClassifier(n_estimators=50,
                                                    random_state=12345)]}],
             scoring='f1')

Теперь выберем лучшую модель и посмотрим ее метрику.

In [ ]:
print('Лучшая модель и её параметры:\n\n', grid.best_estimator_)
print ('Метрика F1 лучшей модели на кросс-валидации:', grid.best_score_)

Лучшая модель и её параметры:

 Pipeline(steps=[('vect', TfidfVectorizer()),
                ('models',
                 LogisticRegression(C=5, penalty='l1', random_state=12345,
                                    solver='liblinear'))])
Метрика F1 лучшей модели на кросс-валидации: 0.7892618469027409


Лучшей моделью получилась логистическая регрессия со сложностью модели 5.

### Анализ моделей

Для анализа моделей выведем таблицу с лучшими результатами моделей. Для этого сгруппируем результаты по названию модели и максимальному значению F1. Оставим в таблице только гиперпараметры лучших экземпляров моделей, а также время обучения и время предсказания.

In [ ]:
res = pd.DataFrame(grid.cv_results_)
res.mean_test_score = res.mean_test_score
res['param_models'] = res['param_models'].astype('category')
t=res.pivot_table(index='param_models', values='mean_test_score', aggfunc='min')
t.columns = ['max_f1']
res1=t.merge(res, left_on='max_f1', right_on='mean_test_score', how='left')
res1.drop(['std_fit_time', 'std_score_time', 'params',
       'split0_test_score', 'split1_test_score', 'split2_test_score',
       'mean_test_score', 'std_test_score', 'rank_test_score'], axis=1)

,max_f1,mean_fit_time,mean_score_time,param_models,param_models__n_neighbors,param_models__C,param_models__penalty
0,0.204299,4.908387,295.683855,KNeighborsClassifier(),4,NaN,NaN
1,0.776747,21.500737,2.428766,"LogisticRegression(random_state=12345, solver=...",NaN,5,l2
2,0.718103,78.234354,4.328923,"LGBMClassifier(n_estimators=50, random_state=1...",NaN,NaN,NaN


Видно, что на обучающей выборке модель логистической регрессии ненамного лучше, чем LightGBM, но по времени обучения значительно быстрее.

### Тестирование

Посмотрим, какой результат покажет лучшая модель на тестовой выборке.

In [ ]:
pred = grid.best_estimator_.predict(X_test)
print("F1 тестовой выборки:", f1_score(y_test, pred))

F1 тестовой выборки: 0.8034471329134902


Значение метрики F1 получилось 0.8, что укладывается в критерии заказчика, так что будем рекомендовать заказчику именно эту модель.

***Вывод*** Мы провели обучение трех моделей с разными гиперпараметрами. После кросс-валидации лучшей моделью логистическая регрессия со сложностью модели 5. Мы проанализировали все модели, выяснили, что кроме того, что у логистической регрессии лучшая метрика, она еще и потратила меньше всего времени на обучение. На тестовой выборке значение метрики F1 у лучшей модели получилось 0.8, что укладывается в критерии заказчика.

## Выводы

Мы провели работу по обучению модели, которая классифицируют комментарии на позитивные и негативные. На пути к выполнению этой задачи мы провели следующую работу:
1. Загрузили датасет, общая информация показала, что в нем 152292 текста, дубликатов и пропусков нет. Комментарии на английском языке.
2. На этапе подготовки данных мы проанализировали дисбаланс классов целевого признака. Кроме того, провели обработку текстов: очистили от лишней информации, провели лемматизацию. Затем разбили датасет на тренировочную и тестовую выборки.
3. Провели обучение трех моделей с разными гиперпараметрами, предварительно получив признаки с помощью TF-IDF . После кросс-валидации лучшей моделью логистическая регрессия со сложностью модели 5. Проанализировали все модели, выяснили, что кроме того, что у логистической регрессии лучшая метрика, она еще и потратила меньше всего времени на обучение. На тестовой выборке значение метрики F1 у лучшей модели получилось 0.8, что укладывается в критерии заказчика.

Таким образом, для выявления токсичных комментариев мы можем рекомендовать заказчику модель логистической регрессии.